In [1]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt
import pydot

Using TensorFlow backend.


In [2]:
%matplotlib inline
import pandas as pd
import os,shutil,math,scipy,cv2
import numpy as np
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.metrics import confusion_matrix,roc_curve,auc
from IPython.display import display
from PIL import Image
from PIL import Image as pil_image
from time import time
from PIL import ImageDraw
from glob import glob
from tqdm import tqdm
from skimage.io import imread
from IPython.display import SVG

from scipy import misc,ndimage
from scipy.ndimage.interpolation import zoom
from scipy.ndimage import imread

from keras import backend as K
from keras import layers
from keras.preprocessing.image import save_img
from keras.utils.vis_utils import model_to_dot
from keras.applications.vgg19 import VGG19,preprocess_input
from keras.applications.xception import Xception
from keras.applications.nasnet import NASNetMobile
from keras.models import Sequential,Input,Model
from keras.layers import Dense,Flatten,Dropout,Concatenate,GlobalAveragePooling2D,Lambda,ZeroPadding2D
from keras.layers import SeparableConv2D,BatchNormalization,MaxPooling2D,Conv2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam,SGD
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard,CSVLogger,ReduceLROnPlateau,LearningRateScheduler

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True,validation_split=0.3)

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [5]:
training_set = train_datagen.flow_from_directory('seg_train',target_size=(224,224),batch_size=8,class_mode = 'categorical')

Found 14034 images belonging to 6 classes.


In [6]:
test_set = test_datagen.flow_from_directory('seg_test',target_size=(224,224),batch_size=8,class_mode = 'categorical')

Found 3000 images belonging to 6 classes.


In [7]:
training_set.target_size,test_set.target_size

((224, 224), (224, 224))

In [8]:
train_size = training_set.n
test_size = test_set.n

train_size,test_size

(14034, 3000)

In [10]:
def ConvBlock(model, layers, filters,name):
    for i in range(layers):
        model.add(Conv2D(filters, (3, 3), activation='relu',name=name))
        model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
def vgg16():
    model = Sequential()
    model.add(Lambda(lambda x: x, input_shape=(224, 224, 3)))
    ConvBlock(model, 1, 64,'block_1')
    ConvBlock(model, 1, 128,'block_2')
    ConvBlock(model, 1, 256,'block_3')
    ConvBlock(model, 1, 512,'block_4')
    model.add(Flatten())
    model.add(Dense(1024,activation='relu'))
    model.add(Dense(4096,activation='relu'))
    model.add(Dense(1000,activation='softmax'))
    return model

model = vgg16()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_2 (Lambda)            (None, 224, 224, 3)       0         
_________________________________________________________________
block_1 (Conv2D)             (None, 222, 222, 64)      1792      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 111, 111, 64)      0         
_________________________________________________________________
block_2 (Conv2D)             (None, 109, 109, 128)     73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 54, 54, 128)       0         
_________________________________________________________________
block_3 (Conv2D)             (None, 52, 52, 256)       295168    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 26, 26, 256)       0         
__________

In [11]:
opt = SGD(lr=1e-4,momentum=0.99)
opt1 = Adam(lr=2e-4)

model.compile(
    loss='binary_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

In [12]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
from PIL import Image
history = model.fit_generator(training_set,steps_per_epoch = train_size,epochs = 5,validation_data = test_set,validation_steps  = test_size)

Epoch 1/40


In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig('modelacc')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig('modelloss')
plt.show()